In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    


In [2]:
# Read Data
other_products_contracts = pd.read_pickle('../data/pickle/'+cfg['other_contracts_file']+'.pickle')

df = other_products_contracts
del other_products_contracts

df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-03-18,DLT1,2015-03-18,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Clinical Chemistry,AG-NOAM-CAN-NEW-WEST0-15D,NaN,N.A,0-00-00,Educational,2015-03-18,2015-03-18,2015.00,INVOICED
1,2015-03-18,DLT1,2015-03-18,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Rodak's Hematology,AG-NOAM-CAN-NEW-WEST0-15D,NaN,N.A,0-00-00,Educational,2015-03-18,2015-03-18,2015.00,INVOICED
2,2015-04-10,DLT1,2015-04-10,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Linne & Ringsrud's Clinical Laboratory Science,AG-NOAM-CAN-NEW-WEST0-15D,NaN,N.A,0-00-00,Educational,2015-04-10,2015-04-10,2015.00,INVOICED
3,2015-07-28,DLT1,2015-07-28,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Echocardiography for the Neonatologist,AG-NOAM-CAN-NEW-WEST0-15D,NaN,N.A,0-00-00,Educational,2015-07-28,2015-07-28,2015.00,INVOICED
4,2015-03-03,DLT1,2015-03-03,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Epidemiology,AG-NOAM-CAN-NEW-WEST0-15D,NaN,N.A,0-00-00,Educational,2015-03-03,2015-03-03,2015.00,INVOICED


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00'], np.nan, inplace=True)
df

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-03-18,DLT1,2015-03-18,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Clinical Chemistry,AG-NOAM-CAN-NEW-WEST0-15D,NaN,NaN,NaN,Educational,2015-03-18,2015-03-18,2015.00,INVOICED
1,2015-03-18,DLT1,2015-03-18,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Rodak's Hematology,AG-NOAM-CAN-NEW-WEST0-15D,NaN,NaN,NaN,Educational,2015-03-18,2015-03-18,2015.00,INVOICED
2,2015-04-10,DLT1,2015-04-10,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Linne & Ringsrud's Clinical Laboratory Science,AG-NOAM-CAN-NEW-WEST0-15D,NaN,NaN,NaN,Educational,2015-04-10,2015-04-10,2015.00,INVOICED
3,2015-07-28,DLT1,2015-07-28,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Echocardiography for the Neonatologist,AG-NOAM-CAN-NEW-WEST0-15D,NaN,NaN,NaN,Educational,2015-07-28,2015-07-28,2015.00,INVOICED
4,2015-03-03,DLT1,2015-03-03,0,0.00,A&G_NOAM,NaN,New,United States,HS,...,Epidemiology,AG-NOAM-CAN-NEW-WEST0-15D,NaN,NaN,NaN,Educational,2015-03-03,2015-03-03,2015.00,INVOICED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11065786,2016-12-31,1-10933468025,2016-08-01,0,152.97,A&G_MAEU,New Sale,New,Ireland,A&G,...,Glucose Intake and Utilization in Pre-Diabetes...,AG-MAEU-North_Europe0-02T,New,Complete,2016-07-25,Region MAEU Inside Sales,2016-12-31,2016-08-01,nan,INVOICED
11065787,2016-12-31,1-10933468025,2016-08-01,0,169.58,A&G_MAEU,New Sale,New,Ireland,A&G,...,"Modulation of Sleep by Obesity, Diabetes, Age,...",AG-MAEU-North_Europe0-02T,New,Complete,2016-07-25,Region MAEU Inside Sales,2016-12-31,2016-08-01,nan,INVOICED
11065788,2016-12-31,1-10933468025,2016-08-01,0,110.28,A&G_MAEU,New Sale,New,Ireland,A&G,...,Molecular Nutrition and Diabetes,AG-MAEU-North_Europe0-02T,New,Complete,2016-07-25,Region MAEU Inside Sales,2016-12-31,2016-08-01,nan,INVOICED
11065789,2016-12-31,1-10933468025,2016-08-01,0,148.23,A&G_MAEU,New Sale,New,Ireland,A&G,...,New Transcription Factors and Their Role in Di...,AG-MAEU-North_Europe0-02T,New,Complete,2016-07-25,Region MAEU Inside Sales,2016-12-31,2016-08-01,nan,INVOICED


In [4]:
df['Sales Type'].value_counts(dropna=False)

NaN                  10097900
New                    498478
Active Renewal         187118
Standard Renewal       129593
Evergreen Renewal       96633
Cancelled               23501
LF                      11036
PF                       9350
Merged                   8841
GR                       3175
LC                        140
RC                         24
PC                          2
Name: Sales Type, dtype: int64

In [5]:
df['Status'].value_counts(dropna=False)

NaN                             10110867
Complete                          878915
Cancelled Complete                 22782
With Customer                      20386
With Sales Rep                     10787
Merged Complete                     8675
Approved                            6015
With Invoicing Group ON HOLD        4580
Dutch-Based Signed                   949
With Product Organization            722
With E-Delivery                      663
With E-Delivery ON HOLD              150
Initial Review                       115
With Invoicing Group                  63
Under Approval                        61
With CIM                              37
CRM Auto Renewed                      20
Cancelled                              4
Name: Status, dtype: int64

### There are too many NAs in Status and Status payments we will delete these rows

In [6]:
#Dropping rows with missing value rate higher than threshold
df = df.loc[~df['Sales Type'].isnull()]


### Summary of the dataframe

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
14,Parent Agreement Number,"[[nan, 1-7742179017, 1-7374363223, 1-539663949...",91952,object,56.45
24,Renewal Exp Complete Date,"[[nan, 2016-02-12, 2016-03-10, 2016-01-04, 201...",2033,object,52.18
34,Subscription Start Year,"[[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020...",7,float64,16.57
16,Payment Term Description,"[[nan, Net Due in 30 days After Invoice Date, ...",102,object,4.93
17,Payment Term Type,"[[nan, NET, SPL/MTH, AS_IS_DATE]]",4,object,4.93
15,Payment Term,"[[nan, 30 NET, 60 NET, SPL4/MTH12, 120 NET, 15...",105,object,4.93
11,Invoice Date,"[[2015-03-02, 2014-12-02, 2015-01-19, 2015-05-...",1734,object,3.77
12,Invoice Num,"[[3652652Y, 3526306Y, 3581524Y, 3745762Y, 3569...",153712,object,3.77
6,Business Indicator,"[[nan, Cancelled, Renewal, New Sale, Credit]]",5,object,2.45
30,Status Change Date,"[[nan, 2015-05-04, 2015-02-12, 2015-03-10, 201...",1556,object,2.45


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In this case, manually ignore Renewal Exp Complete Date.

In [8]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove borderline case
df = df.drop(columns='Renewal Exp Complete Date')

-------------------- Dropping Columns with missing data --------------------
Following 0 columns have missing data over the threshold and will be removed
Index([], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
14,Parent Agreement Number,"[[nan, 1-7742179017, 1-7374363223, 1-539663949...",91952,object,56.45
33,Subscription Start Year,"[[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020...",7,float64,16.57
16,Payment Term Description,"[[nan, Net Due in 30 days After Invoice Date, ...",102,object,4.93
17,Payment Term Type,"[[nan, NET, SPL/MTH, AS_IS_DATE]]",4,object,4.93
15,Payment Term,"[[nan, 30 NET, 60 NET, SPL4/MTH12, 120 NET, 15...",105,object,4.93
11,Invoice Date,"[[2015-03-02, 2014-12-02, 2015-01-19, 2015-05-...",1734,object,3.77
12,Invoice Num,"[[3652652Y, 3526306Y, 3581524Y, 3745762Y, 3569...",153712,object,3.77
6,Business Indicator,"[[nan, Cancelled, Renewal, New Sale, Credit]]",5,object,2.45
29,Status Change Date,"[[nan, 2015-05-04, 2015-02-12, 2015-03-10, 201...",1556,object,2.45
28,Status,"[[nan, Cancelled Complete, Complete, Merged Co...",17,object,2.45


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

In this case, only Payment Term Type needs to be imputed. Since it's a categprical variable, will we impute with the most frequently occuring value.

In [10]:
# #Filling all missing values with 0
# # data = data.fillna(0)

# #Filling missing values with the most frequest values
# df['Payment Term Type'].fillna(
#     df['Payment Term Type'].value_counts().idxmax(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [11]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['Bookigns - Committed Print(Rep)'])

Dropping outliers for Bookings - Final Net Price - Agent Discount Amount(Rep) upper limit = 22745.649380870447 and lower limit = 0.0
Dropping outliers for Subscription Start Year upper limit = 2020.0 and lower limit = 2015.0


# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [12]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
3,Bookigns - Committed Print(Rep),[[0]],1,int64,0.00
7,Calculated New/Renewal,"[[Renewal, New]]",2,object,0.00
22,Product Revenue Type,"[[Recurring, One Off, One Off/Recurring]]",3,object,0.00
9,Division,"[[HS, Corporate, A&G]]",3,object,0.00
17,Payment Term Type,"[[nan, NET, AS_IS_DATE, SPL/MTH]]",4,object,5.71
33,Subscription Start Year,"[[2016.0, 2017.0, 2018.0, 2019.0]]",4,float64,0.00
6,Business Indicator,"[[nan, Renewal, Cancelled, New Sale, Credit]]",5,object,3.93
34,WIP Flag,"[[INVOICED, MERGED, WIP_PPL, DELETED, WIP, CNI]]",6,object,0.00
5,Business Division (Agreement SIS),"[[HS EMEA/LA, A&G_NOAM, Corporate, A&G_MAEU, A...",9,object,0.00
23,RSO,"[[EMEALA, NOAM, APAC, MAEU, EMLA, KEYA, RSS, C...",10,object,0.00


In [14]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [15]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [16]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
3,Bookigns - Committed Print(Rep),[[0]],1,int64,0.00
7,Calculated New/Renewal,"[[Renewal, New]]",2,object,0.00
22,Product Revenue Type,"[[Recurring, One Off, One Off/Recurring]]",3,object,0.00
9,Division,"[[HS, Corporate, A&G]]",3,object,0.00
17,Payment Term Type,"[[nan, NET, AS_IS_DATE, SPL/MTH]]",4,object,5.71
33,Subscription Start Year,"[[2016.0, 2017.0, 2018.0, 2019.0]]",4,float64,0.00
6,Business Indicator,"[[nan, Renewal, Cancelled, New Sale, Other]]",5,object,3.93
34,WIP Flag,"[[INVOICED, MERGED, WIP_PPL, DELETED, WIP, CNI]]",6,object,0.00
5,Business Division (Agreement SIS),"[[HS EMEA/LA, A&G_NOAM, Corporate, A&G_MAEU, A...",9,object,0.00
23,RSO,"[[EMEALA, NOAM, APAC, MAEU, EMLA, KEYA, RSS, C...",10,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [17]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [18]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 3 features - **********************
 Bookigns - Committed Print(Rep) standardized to Bookigns - Committed Print(Rep)
 Bookings - Final Net Price - Agent Discount Amount(Rep) standardized to Bookings - Final Net Price - Agent Discount Amount(Rep)
 Subscription Start Year standardized to Subscription Start Year


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [19]:
# print(columns_to_bin)

In [20]:
# # Specify Columns to encode
# columns_to_exclude = ['Product Line Level 3','Payment Term Description','Payment Term', 
#                       'Business Division (Agreement SIS)', 'Product Line Level 4']
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [21]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [22]:
# df.shape

In [23]:
df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
2179464,2016-12-31,DLF1,2016-01-01,nan,-0.53,HS EMEA/LA,NaN,Renewal,Netherlands,HS,...,TRAITE COSMETOLOGIE ET DERMATOLOGIE ESTHETIQUE,HS Europe North,PF,NaN,NaN,HS France,2016-12-31,2016-01-01,-1.24,INVOICED
2179683,2016-12-31,DLF1,2016-01-01,nan,-0.49,HS EMEA/LA,NaN,Renewal,France,HS,...,TRAITE TC APPAREIL DIGESTIF,HS-EMEALA-France00000-06D,PF,NaN,NaN,HS France,2016-12-31,2016-01-01,-1.24,INVOICED
2180080,2016-12-31,DLF1,2016-01-01,nan,-0.50,HS EMEA/LA,NaN,Renewal,France,HS,...,TRAITE MEDECINE URGENCE,HS-EMEALA-France00000-03D,PF,NaN,NaN,HS France,2016-12-31,2016-01-01,-1.24,INVOICED
2180087,2016-12-31,DLF1,2016-01-01,nan,-0.40,HS EMEA/LA,NaN,Renewal,France,HS,...,EM CONSULTE,HS-EMEALA-France00000-01D,PF,NaN,NaN,HS France,2016-12-31,2016-01-01,-1.24,INVOICED
2180088,2016-12-31,DLF1,2016-01-01,nan,-0.47,HS EMEA/LA,NaN,Renewal,France,HS,...,TRAITE TC ORTHOPEDIE-TRAUMATOLOGIE,HS-EMEALA-France00000-01D,PF,NaN,NaN,HS France,2016-12-31,2016-01-01,-1.24,INVOICED


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [24]:
# convert date columns to date data type

date_columns = ['Agreement Start Date', 'Agreement End Date','Subscription Start Date', 'Subscription End Date','Invoice Date', 'Status Change Date']

In [25]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [26]:
# calcualte new variable subscription_length and agreement_length
df['subscription_length'] = months_between_columns(df['Subscription End Date'],df['Subscription Start Date'])

In [27]:
df['agreement_length'] = months_between_columns(df['Agreement End Date'],df['Agreement Start Date'])

In [28]:
 df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag,subscription_length,agreement_length
2179464,2016-12-31,DLF1,2016-01-01,nan,-0.53,HS EMEA/LA,NaN,Renewal,Netherlands,HS,...,PF,NaN,NaT,HS France,2016-12-31,2016-01-01,-1.24,INVOICED,12.00,12.00
2179683,2016-12-31,DLF1,2016-01-01,nan,-0.49,HS EMEA/LA,NaN,Renewal,France,HS,...,PF,NaN,NaT,HS France,2016-12-31,2016-01-01,-1.24,INVOICED,12.00,12.00
2180080,2016-12-31,DLF1,2016-01-01,nan,-0.50,HS EMEA/LA,NaN,Renewal,France,HS,...,PF,NaN,NaT,HS France,2016-12-31,2016-01-01,-1.24,INVOICED,12.00,12.00
2180087,2016-12-31,DLF1,2016-01-01,nan,-0.40,HS EMEA/LA,NaN,Renewal,France,HS,...,PF,NaN,NaT,HS France,2016-12-31,2016-01-01,-1.24,INVOICED,12.00,12.00
2180088,2016-12-31,DLF1,2016-01-01,nan,-0.47,HS EMEA/LA,NaN,Renewal,France,HS,...,PF,NaN,NaT,HS France,2016-12-31,2016-01-01,-1.24,INVOICED,12.00,12.00


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [29]:
# @TODO
# check the columns use of 'Status Change Date', 'Renewal Exp Complete Date'
# check 'Bookigns - Committed Print(Rep)'

columns_to_remove = ['HQ SIS Id (Agreement SIS)', 'Name  (Agreement SIS)', 'Invoice Num', 'Invoice Date', 'Payment Term Description', 
                     'Status Change Date', 'Product Line Level 3', 'Product Line Level 4', 'Subscription Start Year', 'Bookigns - Committed Print(Rep)']

In [30]:
df = df.drop(columns_to_remove, axis=1)

In [31]:
df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,Parent Agreement Number,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Subregion Grouping,Subscription End Date,Subscription Start Date,WIP Flag,subscription_length,agreement_length
2179464,2016-12-31,DLF1,2016-01-01,-0.53,HS EMEA/LA,NaN,Renewal,Netherlands,HS,NaN,...,TRAITE COSMETOLOGIE ET DERMATOLOGIE ESTHETIQUE,HS Europe North,PF,NaN,HS France,2016-12-31,2016-01-01,INVOICED,12.00,12.00
2179683,2016-12-31,DLF1,2016-01-01,-0.49,HS EMEA/LA,NaN,Renewal,France,HS,NaN,...,TRAITE TC APPAREIL DIGESTIF,HS-EMEALA-France00000-06D,PF,NaN,HS France,2016-12-31,2016-01-01,INVOICED,12.00,12.00
2180080,2016-12-31,DLF1,2016-01-01,-0.50,HS EMEA/LA,NaN,Renewal,France,HS,NaN,...,TRAITE MEDECINE URGENCE,HS-EMEALA-France00000-03D,PF,NaN,HS France,2016-12-31,2016-01-01,INVOICED,12.00,12.00
2180087,2016-12-31,DLF1,2016-01-01,-0.40,HS EMEA/LA,NaN,Renewal,France,HS,NaN,...,EM CONSULTE,HS-EMEALA-France00000-01D,PF,NaN,HS France,2016-12-31,2016-01-01,INVOICED,12.00,12.00
2180088,2016-12-31,DLF1,2016-01-01,-0.47,HS EMEA/LA,NaN,Renewal,France,HS,NaN,...,TRAITE TC ORTHOPEDIE-TRAUMATOLOGIE,HS-EMEALA-France00000-01D,PF,NaN,HS France,2016-12-31,2016-01-01,INVOICED,12.00,12.00


Save clean pickle files

In [32]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['other_contracts_file']+"_for_graphs.pickle")